## **ONLINE stage**

#### LOAD LIBRARIES

In [1]:
# Pulizia
from IPython import get_ipython
get_ipython().magic('clear')
get_ipython().magic('reset -f')

# Import libraries:
import numpy as np
import pandas as pd
import h5py

/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_15729/3705812913.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('clear')
/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_15729/3705812913.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')


#### MATRICES LOADING

In [2]:
# Apri il file in modalità lettura
with h5py.File('../2. DEIM-POD/matrici_RB.h5', 'r') as hf:
    # Accedi ai dataset e ricrea le matrici
    MM_RB = hf['MM_RB'][:]
    FF_RB = hf['FF_RB'][:]
    KK_RB = hf['KK_RB'][:]
    AA_RB = hf['AA_RB'][:]
    ZZ_CO = hf['ZZ_CO'][:]
    II_CO = hf['II_CO'][:]

#### SELECTION OF THE FULL ORDER PARAMETERS 

In [4]:
TIME_DELTA = 10000
#FISSION_RATE = np.random.uniform(low=MIN_FR, high=MAX_FR, size=(1))
FISSION_RATE = 3e19
FISSION_HEAT = 7.27E-18                  #[K/fissions]    = q'''/(rho Cp)            
FISSION_YIELD = 0.24E+00                
SOURCE_T = FISSION_RATE * FISSION_HEAT   
SOURCE_C = FISSION_RATE * FISSION_YIELD  
ALPHA_T = 5.00E-07                       #[m^2 / s]   # = k/(rho Cp)
#LENGTH = np.random.uniform(low=MIN_LL, high=MAX_LL, size=(1))
LENGTH = 1e-03
#RADIUS = np.random.uniform(low=MIN_RR, high=MAX_RR, size=(1))
RADIUS = 1e-05
#T_BC = np.random.uniform(low=MIN_BC, high=MAX_BC, size=(1))
T_BC = 2000
print("The selected Boundary Temperature is", T_BC, "K")
print("The selected Fission Rate is", FISSION_RATE, "f/s")
print("The selected Grain Length is", LENGTH, "m")
print("The selected Grain Radius is", RADIUS, "m\n")

The selected Boundary Temperature is 2000 K
The selected Fission Rate is 3e+19 f/s
The selected Grain Length is 0.001 m
The selected Grain Radius is 1e-05 m



#### CONSTRUCTION OF THE LHS AND RHS

In [5]:
TT = T_BC + (SOURCE_T * LENGTH**2 / ALPHA_T) * (1 - ZZ_CO**2) / 2

#SCIANTIX userà il modello scelto dall'utente 
WW = 5.0e-08 * np.exp(-40262.0 / (TT));
 
CC_CO = np.hstack((II_CO @ WW / RADIUS**2, II_CO @ WW / LENGTH**2))
 
SS_RB = np.tensordot(CC_CO, KK_RB, axes=[0, 0])
QQ_RB = SOURCE_C * FF_RB
 
LHS_RB_static = MM_RB + TIME_DELTA * SS_RB #matrice + matrice * scalare. Avendo il termine MM_RB al lhs è forward euler, se invece fosse nel RHS sarebbe Backward. 
RHS_RB_static = TIME_DELTA * QQ_RB         #vettore per scalare

In [6]:
print(II_CO[0,1])
print(WW[1])
print(CC_CO[1])

print(SS_RB[0,1])
print(LHS_RB_static[0,1])

-0.8066441654392976
1.25953854547046e-16
-6.935175427394345e-06
8.081481438527624e-06
0.0808148143852822


In [7]:
print(KK_RB[0,0,0])
print(KK_RB[1,0,0])
print(KK_RB[0,1,0])
print(KK_RB[0,0,1])

np.where(KK_RB == 0.021958)


-0.25722857710170016
0.5176075764116836
-0.11730967282402961
-0.11730967282402949


(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

### FORWARD EULER

In [8]:
old_sol_RB = np.zeros((MM_RB.shape[0], 1))
reconstructed_solution = np.zeros((1000))
for ii in range(1000):
    old_sol_RB = np.linalg.solve(LHS_RB_static, RHS_RB_static + MM_RB.dot(old_sol_RB))
    value = np.dot(AA_RB, old_sol_RB)[0]
    reconstructed_solution[ii] = value
    print(reconstructed_solution[ii])

4.714992064352094e+22
8.3625368718144e+22
1.1305285883080921e+23
1.373032396023603e+23
1.5756904057945675e+23
1.7468778538414777e+23
1.892785352211777e+23
2.0181274073833436e+23
2.126564308093653e+23
2.2209774551788522e+23
2.3036600578645632e+23
2.3764543566022092e+23
2.4408528633384163e+23
2.4980743366019084e+23
2.5491215038669612e+23
2.5948253384341858e+23
2.63587929432794e+23
2.6728659632354e+23
2.7062779657293455e+23
2.7365344251662042e+23
2.763994036498317e+23
2.7889654953821132e+23
2.8117158698614198e+23
2.8324773600212915e+23
2.8514527880153942e+23
2.8688200829499252e+23
2.884735965866196e+23
2.8993389948212368e+23
2.912752095365704e+23
2.9250846749944146e+23
2.9364343994798287e+23
2.9468886929544647e+23
2.9565260111006667e+23
2.9654169270157647e+23
2.973625061625243e+23
2.9812078844422538e+23
2.9882174056566877e+23
2.9947007767033383e+23
3.0007008133931007e+23
3.0062564532287935e+23
3.0114031565405575e+23
3.0161732594657344e+23
3.0205962854875254e+23
3.024699221174917e+23
3.028

#### COMPARISON WITH THE FULL ORDER MODEL

In [9]:
full_order= pd.read_csv('../0. full order/Sensitivity/10Layers_5Refine_0.75Transition_4Center.csv')

error = np.abs(full_order['Average dC (atm/m^3)'].iloc[-1] - reconstructed_solution[-1])/full_order['Average dC (atm/m^3)'].iloc[-1]
print(error)

2.942631738411281e-06
